# Google Scholar 
Try avoid robots by explicit clicking on the previously loaded 
page

See: https://academia.stackexchange.com/q/2567

Check for https://github.com/ckreibich/scholar.py/blob/master/scholar.py

See also: lookup fields:
https://scholar.google.co.in/scholar_lookup?title=Estimates+for+the+number+of+sums+and+products+and+for+exponential+sums+in+fields+of+prime+order&author=Jean+Bourgain&author=AA+Glibichuk&author=SERGEI+VLADIMIROVICH+Konyagin&year=2006&doi=10.1112/S0024610706022721&publisher=Oxford+University+Press&journal=Journal+of+the+London+Mathematical+Society&volume=73&issue=2&pages=380-398&hl=en

https://pypi.org/project/googletrans/

## Choose between selenium or requests

In [1]:
url='https://scholar.google.com'
CITES=False
if CITES:
    url=url+'/scholar?cites=16729892078990709043&as_sdt=2005&sciodt=0,5&hl=es'
SELENIUM=True
if SELENIUM:
    #pip3 install webdriver_manager
    from webdriver_manager.firefox import GeckoDriverManager
    from selenium import webdriver
    from pathlib import Path
    home = str(Path.home())
    try:
        browser = webdriver.Firefox(executable_path=
          '/home/restrepo/.wdm/geckodriver/v0.23.0/linux64/geckodriver')
    except:
        browser = webdriver.Firefox(
            executable_path= GeckoDriverManager().install())
    
    browser.get(url)
    #TODO: Authentication:
    # Login page
    # gs_hdr_act_s by id

In [111]:
browser.find_element_by_id("gs_hdr_act_s").click()

In [112]:
browser.find_element_by_id("identifierId").clear()

In [113]:
browser.find_element_by_id("identifierId").send_keys("admin@fisica.udea.edu.co")

In [114]:
browser.find_element_by_class_name("RveJvd").click()

In [115]:
import getpass

In [116]:
password=getpass.getpass()

········


In [117]:
browser.find_element_by_class_name("whsOnd").send_keys(password)

In [118]:
browser.find_element_by_class_name("RveJvd").click()

In [3]:
if not SELENIUM:
    import requests
    headers_Get = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }
    r=requests.Session()
    browser=None

In [6]:
import scholarly

In [7]:
q='An examination of the Degtjareff method for determining soil organic matter, and a proposed modification of the chromic acid titration method'
search_query = scholarly.search_pubs_query(q)

In [8]:
d=next(search_query)

In [10]:
d.id_scholarcitedby

'6597111764647197210'

In [ ]:
>>> search_query = scholarly.search_pubs_query('Perception of physical stability and center of mass of 3D objects')
>>> print(next(search_query))

In [148]:
d.bib

{'author': 'CM Jiménez-Yepes and JL Londoño-Fernández\xa0- Stroke and 2008',
 'eprint': 'https://scholar.google.comhttps://pdfs.semanticscholar.org/02be/d7dc396ec984e22661c5f385d7088b083a43.pdf',
 'title': 'Risk of aneurysmal subarachnoid hemorrhage: the role of confirmed hypertension'}

In [75]:
cites_query=d.get_citedby()

In [76]:
dd=next(cites_query)

In [77]:
dd.bib

{'abstract': 'A synergistic effect of alcohol and hypertension has been suggested to increase the risk for stroke. However, the contribution of alcohol-induced hypertension to stroke morbidity and mortality may be greater than observed, because the effects of different drinking patterns\xa0…',
 'author': 'M Hillbom and P Saloheimo and S Juvela\xa0- Current hypertension reports and 2011',
 'eprint': 'https://scholar.google.comhttps://www.researchgate.net/profile/Seppo_Juvela/publication/49842166_Alcohol_Consumption_Blood_Pressure_and_the_Risk_of_Stroke/links/54a877120cf267bdb90b3431.pdf',
 'title': 'Alcohol consumption, blood pressure, and the risk of stroke',
 'url': 'https://link.springer.com/article/10.1007/s11906-011-0194-y'}

In [3]:
#import googlescholar as gs
#%%writefile ../cienciometria/googlescholar.py
import Levenshtein
import re
from bs4 import BeautifulSoup
import requests
import scholarly

def scholarly_to_gs(d):
    if not d:
        return d
    gsr={}
    try:
        gsr['cites']=d.citedby
        gsr['cites_link']=d.id_scholarcitedby
    except AttributeError:
        gsr['cites']=0
        gsr['cites_link']=''
    gsr['title']=d.bib.get('title')
    gsr['ref']=d.bib.get('author').replace(' and',',')
    gsr['authors']=gsr['ref'].split('-')[0].strip()
    gsr['PDF']=d.bib.get('eprint')
    
    try:
        jy=gsr['ref'].split('-')[1].strip()
        gsr['Journal']=jy.split(',')[0]
        try:
            gsr['Year']=eval(jy.split(',')[1])
        except:
            gsr['Year']=-1
    except:
        gsr['Journal']=''

    gsr['abstract']=d.bib.get('abstract')
    return gsr

def firefox_get(url,browser=browser):
    try:
        if browser:
            if url.find('scholar_lookup?doi')==-1:
                q=url.split('=')[-1]
                browser.find_element_by_id("gs_hdr_tsi").clear()
                browser.find_element_by_id("gs_hdr_tsi").send_keys(q)
                browser.find_element_by_id('gs_hdr_tsb').click()
            else:
                browser.get(url)
        else:
            rget=r.get(url,headers=headers_Get)
    except:
        return 'FAILED'
    
    if browser:
        html=browser.page_source
    else:
        html = rget.text
    if html.lower().find('gs_captcha_f')>-1:
        input('check robots')
    #if html.find('Sorry')>-1:
    #    input('check sorry')

    return html

def get_google_scholar(record):
    '''
    Analyise the BeautifulSoup record for an article 
    in Google Scholar.
    Output is a Python dictionary with keys: 
    'title', 'authors','profiles','Jornal','Year',
    'abstract','cites','cites_link'
    '''
    import random
    import time
    gsr={}
    try:
        cites=record.find_all('a',{"href":re.compile( "/scholar\?cites*" )})[0]
        try:
            gsr['cites']=int( cites.text.split()[-1] )
            gsr['cites_link']=cites.attrs.get('href')
        except:
            gsr['cites']=0
    except:
        cites=None

    # Title
    try:
        #The .split('XXX')[-1]  does not afect the result when .text does no contains 'XXX'
        tc=record.find_all('h3',{"class":"gs_rt"})[0].text.split('[CITATION][C] ')[-1]
    except:
        tc=''

    gsr['title']=tc.strip().split('[HTML][HTML] ')[-1].split(
                                  '[PDF][PDF] '  )[-1]
    
    # Explore authors, google scholar profile, Journal and Year
    gpa=None
    try:
        gpa=record.find_all('div',{"class":"gs_a"})[0]
        gsr['ref']=gpa.text.strip()
        gsr['authors']=gpa.text.split('-')[0].strip()
        try:
            jy=gpa.text.split('-')[1].strip()
            gsr['Journal']=jy.split(',')[0]
            try:
                gsr['Year']=eval(jy.split(',')[1])
            except:
                gsr['Year']=-1
        except:
            gsr['Journal']=''
    except:
        gsr['authors']=''
        gsr['ref']=''

    #Abstract:
    try:
        gsr['abstract']=record.find_all('div',{'class':'gs_rs'})[0].text.replace('\xa0…','')
    except:
        gsr['abstract']=''
    # citations
    if gpa:
        lpr=gpa.find_all("a",{ "href":re.compile("/citations\?user=*")   } )
        prf={}
        for pr in lpr:
            prf[ pr.text ]=pr.attrs.get('href').split('?')[-1].split('&')[0].split('user=')[-1]
        gsr['profiles']=prf
    
    time.sleep( random.randint(1,3)  ) # avoid robots
    return gsr

def request_google_scholar_url(url):
    return requests.get(url)

def google_scholar_page(html):
    '''
    Convert a Google Scholar page into a list
    of dictionaries with metadata info
    '''
    if html.lower().find('gs_captcha_f')>-1:
        input('check robots')
   
    soup = BeautifulSoup(html, "html.parser")
    rgs=soup.find_all('div', {'class':'gs_ri' })

    citations=[]
    for record in rgs:
        citations.append( get_google_scholar(record) )
        
    return citations

def google_scholar_query(title='relativity theory',author="A Einstein",
                         DOI=None,DEBUG=False,Scholarly=False):
    '''
    Search Google scholar for similarity in title and author.
    Only the first result is analized. The output includes 
    a quality measurements between the query and the results 
    Output is a Python dictionary with keys: 
    'title', 'authors','profiles','cites','cites_link',
    'quality_title','quality_author'
    '''
    gs={}
    # + → %2B in query formula:
    baseurl='https://scholar.google.com/'
    if DOI:
        baseurl=baseurl+'scholar_lookup?doi='
        q='{}'.format(DOI)
    else:
        baseurl=baseurl+'scholar?q='
        q='"{}"%2B{}'.format(title,author)
    
    url=baseurl+q
    if DEBUG:
        print(url)   
    
    #s = requests.Session()
    if not Scholarly:
        rtext=firefox_get(url)

        #soup = BeautifulSoup(r.text, "html.parser")
        soup = BeautifulSoup(rtext, "html.parser")


        # Main contents:
        rgs=soup.find_all('div', {'class':'gs_ri' })

        try:
            record=rgs[0]
        except IndexError:
            #exit if record not found and returns empty dictionary
            return gs
    
        gs.update(get_google_scholar(record))
    else:
        search_query = scholarly.search_pubs_query(q)

        try:
            d=next(search_query)
        except StopIteration:
            return {}     
        gs=scholarly_to_gs(d)
    
    #Check if author is in authors list
    if gs and author:
        sau=0
        for a in gs['authors'].split(','):
            saun=Levenshtein.ratio(author.lower(),a.lower().strip())
            if saun>sau:
                sau=saun
                
        gs['quality_author']=round(sau,2)
    else:
        gs['quality_author']=-1 #-1 means not checked
        
    if gs and title:
        gs['quality_title']=round( Levenshtein.ratio(
                   title.lower(),gs['title'].lower() ),2 )
    else:
        gs['quality_title'] =-1 #-1 means not checked
        
    #EXTRA FIELDS:
    #PDF
    if not Scholarly:
        try:
            gs['PDF']=soup.find_all('div',
                        {"class":"gs_or_ggsm"})[0].find_all('a')[0].get("href")
        except:
            gs['PDF']=''


    #if DEBUG:
    #    return gs,record
    return gs

def get_cites_refs(browser,url,maxcites=65,t=60):
    import random
    import time
    url='https://scholar.google.com'+url
    browser.get(url)
     
    endpage=int(maxcites/10)+1
    refs=''
    
    kk=google_scholar_page( browser.page_source )
    try:
        refs=refs+'\n'.join( list((pd.DataFrame(kk)['title']+'; '
                                  +pd.DataFrame(kk)['ref']).values) )+'\n' 
    except:
        refs=''
    
    
    for i in range(endpage):
        try:
            browser.find_element_by_class_name('gs_ico_nav_next').click()
            kk=google_scholar_page( browser.page_source )
            try:
                refs=refs+'\n'.join( list( (pd.DataFrame(kk)['title']+'; '
                                +pd.DataFrame(kk)['ref']).values ) )+'\n' 
            except:
                refs=''
        except:
            break
            
    time.sleep(random.uniform(0.9*t,1.1*t))
    return refs

In [4]:
import wosplus as wp

In [5]:
%%writefile drive.cfg
[FILES]
DOIS.xlsx               = 1bikNT7Gmp4G7dfeMuGsF-az7D8lskK0O
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj
oaudea.xlsx             = 1CcwobiEFACIbffNzNdLxpdxQukr8cZ5x
datos1.csv              = 11CyLRZZwVbgw6YAC-igRJ3mrkIwk0aaiXnd-EOofYTI

Overwriting drive.cfg


In [6]:
oa=wp.wosplus('drive.cfg')

In [85]:
oaudea=oa.read_drive_excel('oaudea.xlsx')

In [86]:
oaudea=wp.fill_NaN(oaudea)

In [7]:
import time
import random
import pandas as pd

In [8]:
datos=pd.read_json('md.json').sort_index().reset_index(drop=True)

In [9]:
datos.shape

(1767, 5)

In [10]:
datos=datos[~datos.z_authors.isnull()]

In [11]:
datos=datos[~datos.z_authors.str[0].apply(lambda x: x.get('family')).isnull()]#+' '+x.get('family'))
datos=datos[~datos.z_authors.str[0].apply(lambda x: x.get('given')).isnull()]#+' '+x.get('family'))

In [12]:
datos['name_simple']=datos.z_authors.str[0].apply(lambda x: x.get('given')+' '+x.get('family'))
datos['title_simple']=datos.title

In [13]:
#JUST START AFTER FINISH REDALYC
nini=0# initial doi
n=1506 # Total of DOIs
T=12 #hours of search
t=T/n*3600 # [s] query time
t=60
day=24*3600 #s
mintime=0.9*t*n # [s] minimal time search
wait=30#day-mintime # maximum wait

In [14]:
nini=593

In [ ]:
Scholarly=False
name_simple='name_simple'
doi='doi'
title_simple='title_simple'
title='title'
dfgs=pd.DataFrame()
for ii in datos[nini:].index:
    print(ii,datos.loc[ii,doi])
    #gsd=google_scholar_query(DOI=doi)
    gsd=google_scholar_query(DOI=datos.loc[ii,doi],
                     title =datos.loc[ii,title_simple],
                     author=datos.loc[ii,name_simple],Scholarly=Scholarly)
    if not gsd:
        time.sleep(random.uniform(0.9*t,1.1*t))
        gsd=google_scholar_query(title =datos.loc[ii,title_simple],
                     author=datos.loc[ii,name_simple],Scholarly=Scholarly)
    gsd['old_title']=datos.loc[ii,title]
    gsd['DOI']=datos.loc[ii,doi]
    dfgs=dfgs.append(gsd,ignore_index=True )
    dfgs.to_json('gas.json')
    time.sleep(random.uniform(0.9*t,1.1*t))

595 10.1007/jhep10(2015)063
check robots
596 10.3989/revmetalm.053
597 10.1371/journal.pgen.1005493
598 10.1186/s12977-015-0206-0
599 10.1186/s12936-015-0887-9
600 10.1186/s13071-015-1100-2
601 10.1371/journal.pone.0137619
602 10.1371/journal.pntd.0004045
603 10.15446/abc.v20n3.49373
604 10.1371/journal.pone.0136292
605 10.1186/s12864-015-1678-y
606 10.1016/j.ijid.2015.05.023
607 10.3390/toxins7082872
608 10.1371/journal.pone.0132811
609 10.1371/journal.pone.0133683
610 10.1007/jhep07(2015)096
611 10.1038/srep11899
612 10.15446/caldasia.v37n2.53558
613 10.11144/javeriana.m8-16.mmpe
614 10.4314/tjpr.v14i7.3
615 10.4314/tjpr.v14i7.4
616 10.1371/journal.pone.0129113


In [1]:
cp gas.json gas4.json

In [99]:
kk=dfgs[~dfgs.title.isna()]
kk.to_excel('gas3.xlsx',index=False)

In [88]:
ls ga

In [90]:
kk[6:]

,DOI,Journal,PDF,Year,abstract,authors,cites,cites_link,old_title,quality_author,quality_title,ref,title
6,10.1002/ecs2.1698,…,https://scholar.google.comhttps://onlinelibrar...,2017.0,"… School of Forestry, Northern Arizona Univers...","JC Villegas, DJ Law, SC Stark, DM Minor…",2.0,17384195563965748590,Prototype campaign assessment of disturbance-i...,0.71,0.99,"JC Villegas, DJ Law, SC Stark, DM Minor… - …, ...",Prototype campaign assessment of disturbance‐i...
7,10.1186/s13071-017-2031-x,Restrepo…,https://scholar.google.comhttps://parasitesand...,-1.0,… Vectors201710:110. https://doi.org/10.1186/s...,"RJ Vivero, G Cadavid",3.0,6234272273311297660,Molecular detection and identification of Wolb...,0.67,1.00,"RJ Vivero, G Cadavid-Restrepo… - Parasites & …...",Molecular detection and identification of Wolb...
8,10.1186/s12879-017-2258-3,Agudelo…,https://scholar.google.comhttps://bmcinfectdis...,-1.0,… https://doi.org/10.1186/s12879-017-2258-3. ©...,V Velasquez,10.0,17241652251553328440,"Meta-analysis of the utility of culture, biops...",0.59,1.00,"V Velasquez-Agudelo… - BMC infectious …, 2017","Meta-analysis of the utility of culture, biops..."
9,10.1371/journal.pone.0171930,PloS one,https://scholar.google.comhttp://journals.plos...,2017.0,"… Mauricio Rojas. x. Published: February 21, 2...","D Marín, N Marín, H Del Corral, L López…",2.0,9984492423022648444,PPD-induced monocyte mitochondrial damage is a...,0.78,1.00,"D Marín, N Marín, H Del Corral, L López… - Plo...",PPD-induced monocyte mitochondrial damage is a...
10,10.1371/journal.pone.0171191,PloS one,https://scholar.google.comhttp://journals.plos...,2017.0,"… Marcos Lopez,; Antonio Pazin-Filho,; Fabian ...","ME Niño, SE Serrano, DC Niño, DM McCosham…",5.0,13858985965677904829,TIMP1 and MMP9 are predictors of mortality in ...,0.56,0.97,"ME Niño, SE Serrano, DC Niño, DM McCosham… - P...",TIMP1 and MMP9 are predictors of mortality in ...
11,10.3389/fimmu.2017.00136,Jimenez,https://scholar.google.comhttps://www.frontier...,-1.0,"… Original Research ARTICLE. Front. Immunol., ...",W Aguilar,3.0,18043045145402741647,High Expression of Antiviral and Vitamin D Pat...,0.56,1.00,"W Aguilar-Jimenez, I Saulle, D Trabattoni… - F...",High expression of antiviral and vitamin D pat...
12,10.3390/molecules22020211,Molecules,https://scholar.google.comhttps://www.mdpi.com...,2017.0,… ajax/scifeed/subscribe. Quick Links. Open Ac...,"R Brkljača, B Schneider, W Hidalgo, F Otálvaro…",5.0,17653779315016990393,Application of the Crystalline Sponge Method t...,0.80,0.88,"R Brkljača, B Schneider, W Hidalgo, F Otálvaro...",Application of the Crystalline Sponge Method t...
13,10.1186/s12906-017-1562-1,Calderón…,https://scholar.google.comhttps://bmccomplemen...,-1.0,… 17:57. https://doi.org/10.1186/s12906-017-15...,C Gómez,11.0,11129651484715091424,Antiviral effect of compounds derived from the...,0.48,1.00,"C Gómez-Calderón… - BMC …, 2017",Antiviral effect of compounds derived from the...
14,10.3389/fpls.2017.00009,Mora,https://scholar.google.comhttps://www.frontier...,-1.0,… SHARE ON. Open Supplemental Data. Original R...,"Y Madrigal, JF Alzate, N Pabón",5.0,4576764258222618521,Evolution and Expression Patterns of TCP Genes...,0.77,1.00,"Y Madrigal, JF Alzate, N Pabón-Mora - Frontier...",Evolution and expression patterns of TCP genes...
15,10.3989/ajbm.2474,Anales del Jardín Botánico de Madrid,https://scholar.google.comhttps://dialnet.unir...,2017.0,… http://dx.doi.org/10.3989/ajbm.2474 ISSN: 02...,"F González, NP Mora",0.0,,Inflorescence and floral traits of the Colombi...,0.83,0.96,"F González, NP Mora - Anales del Jardín Botáni...",Inflorescence and floral traits of the Colombi...


In [ ]:
google_scholar_query(DOI='10.1161/strokeaha.107.504019',Scholarly=True)

In [29]:
oaudea[oaudea['OA_Green']=='10.1590/s0103-84782009000100018']

,DI,Open_Access,OA_Yellow,OA_Green,ISSNs,OA_Green_Article,ISSNWOS,APC,OAGISSN,APC_USD
606,10.1590/s0103-84782009000100018,10.1590/s0103-84782009000100018,,10.1590/s0103-84782009000100018,0103-8478,,"0103-8478,1678-4596",700BRL,0103-8478,188


In [22]:
oaudea.shape

(7999, 10)

In [14]:
oaudea[oaudea['OA_Yellow']!=''].shape

(1505, 10)

In [26]:
REDALYC=False

In [ ]:
google_scholar_query(DOI='10.1155/2015/978379')

In [28]:
if REDALYC:
    nini=0

In [24]:
oaudea[oaudea['OA_Yellow']!=''].reset_index(drop=True)[1502:1505]

,DI,Open_Access,OA_Yellow,OA_Green,ISSNs,OA_Green_Article,ISSNWOS,APC,OAGISSN,APC_USD
1502,10.17533/udea.ikala.v19n3a05,10.17533/udea.ikala.v19n3a05,10.17533/udea.ikala.v19n3a05,,,,0123-3432,,,0
1503,10.11144/javeriana.mavae9-2.eacv,10.11144/javeriana.mavae9-2.eacv,10.11144/javeriana.mavae9-2.eacv,,,,1794-6670,,,0
1504,10.11144/javeriana.rgyps13-26.eaht,10.11144/javeriana.rgyps13-26.eaht,10.11144/javeriana.rgyps13-26.eaht,,,,1657-7027,,,0


In [22]:
dfgsn=pd.read_json('apc_missing_dois.json')#pd.read_json('yellowno.json')

In [23]:
dfgsn.shape

(1776, 1)

In [ ]:
if REDALYC:
    import unidecode
    datos=pd.read_json('redaylicmiising.json')
    datos['title_simple']=datos.title.str.lower().str.replace(
    '\n',' ').str.replace('\\\\n',' ').map(unidecode.unidecode).str.strip()
    datos['name_simple']=datos.name.str.split("'").str[1].str.strip().str.lower().str.split().str[0].str.strip()+' '+datos.lastname.str.split("'").str[1].str.replace('\\\\n',' '
        ).str.strip().str.lower().str.split(' ').str[0]
    print('SKYP to next cell')

### Search GS for OA

In [ ]:
dfgs=pd.DataFrame()
i=786
#oaudea[oaudea['OA_Yellow']!='']['OA_Yellow']
for doi in dfgsn.DOI.values[nini:]:#nini+n]:
    print(i,doi)
    gsd=google_scholar_query(DOI=doi)
    if not gsd:
        print('WARNING: Not found')
    gsd['DOI']=doi
    dfgs=dfgs.append(gsd,ignore_index=True )
    dfgs.to_json('gas.json')
    time.sleep(random.uniform(0.9*t,1.1*t))
    if i%n==0:
        time.sleep(wait)        
    i=i+1

In [146]:
kk=pd.read_json('gas_0_309.json').append(
   pd.read_json('gas_310_409.json')).append(
   pd.read_json('gas_409_549.json')).append(
   pd.read_json('gas_549_1340.json')).append(
   pd.read_json('gas_missing1.json')).append(
   pd.read_json('gas_missing2.json')).reset_index(drop=True)

In [147]:
kk=kk.fillna('')

In [148]:
kky,kkn=wp.df_split(kk,on='title',Operator='!=',condition='')

In [149]:
dfd=kky.append(kkn).reset_index(drop=True).fillna('')

In [150]:
df=dfd.drop_duplicates('DOI').reset_index(drop=True)

In [151]:
df['cites']=df.cites.replace('',0).astype(int)
df['Year']=df.Year.replace('',-1).astype(int)
df['quality_author']=df.quality_author.replace('',0).astype(float)
df['quality_title']=df.quality_title.replace('',0).astype(float)

In [152]:
df.shape

(1340, 12)

In [153]:
df[df.cites!=0].shape

(1041, 12)

In [154]:
df.to_json('oa_cites.json')

In [155]:
df=pd.read_json('oa_cites.json').reset_index(drop=True)

In [198]:
kk=oaudea[oaudea.DI!=''].rename({'DI':'DOI'},axis='columns')[['DOI','Open_Access','APC']].merge(
    df,on='DOI',how='left').fillna('')

In [201]:
kk[np.logical_and( kk.title=='',kk.APC!='')][['DOI']].to_json('apc_missing_dois.json')

In [189]:
kk[kk.title!='']

,DOI,Journal,Year,abstract,authors,cites,cites_link,profiles,quality_author,quality_title,ref,title
1508,10.7705/biomedica.v32i0.613,Niño,-1,Page 1. 173 Biomédica 2018;38:173-9 Regression...,JA Fernández,1,/scholar?cites=10650095648529382596&as_sdt=200...,{'JA Fernández-Niño': 'IDhzOk4AAAAJ'},0,0,,Reporte estadístico en los análisis de regresi...
1509,10.7705/biomedica.v32i4.421,Biomedica,2012,… Permalink. Biomédica. Print version ISSN 012...,JA Buendía,7,/scholar?cites=1149452076688300574&as_sdt=2005...,{'JA Buendía': 'eWNjMQMAAAAJ'},0,0,,"Attitudes, knowledge and beliefs of patient ab..."
1510,10.7705/biomedica.v31i3.380,…,2011,Page 1. Biomédica 2011 ;31 :392-402 ARTíCULO O...,"ID Flórez, E Ramos, C Bernal, OJ Cuéllar…",2,/scholar?cites=18243831652194567692&as_sdt=200...,{'ID Flórez': '9_05iMYAAAAJ'},0,0,,Encuesta nacional sobre los conocimientos impa...
1512,10.5943/mycosphere/2/6/4,Molano…,-1,Page 1. Mycosphere Doi 10.5943/mycosphere/2/6/...,"NA Herrera, C Rojas, AE Franco",8,/scholar?cites=14376747962915910441&as_sdt=200...,{'C Rojas': 'GGPSTigAAAAJ'},1,1,"NA Herrera, C Rojas, AE Franco-Molano… - …, 20...",Physarella oblonga-centered bioassays for test...
1514,10.7705/biomedica.v37i2.2893,Gómez,-1,… doi: http://dx.doi.org/10.7705/biomedica.v37...,Ó Quirós,2,/scholar?cites=13840368708425379385&as_sdt=200...,"{'N Jaramillo': 'FHr5-hQAAAAJ', 'Ó Quirós-Góme...",1,1,"Ó Quirós-Gómez, N Jaramillo, VM Angulo… - Biom...","Triatoma dimidiata en Colombia; distribución, ..."
1516,10.7705/biomedica.v29i2.25,Carvajal,-1,… Biomédica. 2009;29:232-43. https://doi.org/1...,D Peláez,0,,{},1,1,"D Peláez-Carvajal, NJ Forero, M Escalante-Mora...",Variabilidad genética en regiones codificantes...
1519,10.17533/udea.ef.n56a07,Bermúdez,-1,… Por: Manuel Ernesto Charry Bermúdez Institut...,ME Charry,0,,{},1,1,"ME Charry-Bermúdez - Estudios de Filosofía, 20...",Totalidad y dialéctica. El concepto de totalid...
1521,10.17533/udea.iee.v35n2a12,Botero,-1,… Permalink. Investigación y Educación en Enfe...,FD Tamayo,0,,{},1,1,FD Tamayo-Botero - Investigacion y educacion e...,The Meaning of Assuming Dependency at Home of ...
1526,10.17533/udea.ef.n55a11,Estudios de Filosofía,2017,"… 55, Enero-Junio, 2017, pp. 187-205 Instituto...","VHC Pérez, LF Echeverri, E Zarrazola",0,,{},1,1,"VHC Pérez, LF Echeverri, E Zarrazola - Estudio...",Objetos matemáticos sensibles y objetos Matemá...
1530,10.17533/udea.ikala.v22n01a08,Íkala,-1,… Mailing address: Transversal 51A N.° 67B-90 ...,"ÁM Roldán, OA Peláez",2,/scholar?cites=8955031945038639005&as_sdt=2005...,{'OA Peláez': 'WasxVdYAAAAJ'},1,1,"ÁM Roldán, OA Peláez - Íkala, revista de lengu...",Pertinencia de las Políticas de enseñanza del ...


In [156]:
df=df.sort_values('cites',ascending=False)
df[['cites','DOI','authors','title']]

,cites,DOI,authors,title
1338,5458,10.1038/nature11632,1000 Genomes Project Consortium,"An integrated map of genetic variation from 1,..."
626,4556,10.1007/s11356-016-8253-1,"CG Daughton, TA Ternes",Pharmaceuticals and personal care products in ...
1280,929,10.1038/nm1784,"MA Faghihi, F Modarresi, AM Khalil, DE Wood…",Expression of a noncoding RNA is elevated in A...
1231,874,10.1126/science.1169588,"CG Elsik, RL Tellam, KC Worley",The genome sequence of taurine cattle: a windo...
133,786,10.1038/ng985,"S Kondo, BC Schutte, RJ Richardson, BC Bjork…",Mutations in cause Van der Woude and poplitea...
533,768,10.1016/s0140-6736(11)60278-x,Crash,The importance of early treatment with tranexa...
1228,662,10.1073/pnas.0901438106,"HD Adams, M Guardiola",Temperature sensitivity of drought-induced tre...
159,636,10.1093/oxfordjournals.molbev.a026247,"TJC Anderson, B Haubold, JT Williams…",Microsatellite Markers Reveal a Spectrum of Po...
1157,528,10.1038/nature09084,"G Sansone, F Kelkensberg, JF Pérez",Electron localization following attosecond mol...
973,460,10.1038/nature11258,"D Reich, N Patterson, D Campbell, A Tandon…",Reconstructing native American population history


In [13]:
import numpy as np

In [16]:
dfgsy=kk[np.logical_and( kk.title!='',kk.authors!='')]
tmp=dfgsy.append( kk[np.logical_and( kk.title=='',kk.authors=='')] )

In [17]:
dfgs=tmp.drop_duplicates('DOI')

In [18]:
dfgs.shape

(1333, 12)

In [29]:
dfgsn=dfgs[ np.logical_and(dfgs.title=='',dfgs.authors=='') ].reset_index(drop=True)

In [30]:
dfgsn.shape

(61, 12)

### Search Google cites for Redalyc

In [11]:
oaudea

,APC,APC_USD,DI,ISSNWOS,ISSNs,OAGISSN,OA_Green,OA_Green_Article,OA_Yellow,Open_Access
0,550GBP,713,10.1186/s40104-017-0225-x,2049-1891,2049-1891,2049-1891,10.1186/s40104-017-0225-x,NaN,NaN,10.1186/s40104-017-0225-x
1,4000USD,4000,10.1038/s41531-017-0025-1,2373-8057,2373-8057,2373-8057,10.1038/s41531-017-0025-1,NaN,NaN,10.1038/s41531-017-0025-1
2,1370GBP,1776,10.1186/s12879-017-2929-0,1471-2334,1471-2334,1471-2334,10.1186/s12879-017-2929-0,NaN,NaN,10.1186/s12879-017-2929-0
3,1370GBP,1776,10.1186/s12864-017-4376-0,1471-2164,1471-2164,1471-2164,10.1186/s12864-017-4376-0,NaN,NaN,10.1186/s12864-017-4376-0
4,0NOT,0,10.1140/epjc/s10052-017-5317-4,"1434-6044,1434-6052","1434-6044,1434-6052","1434-6044,1434-6052",10.1140/epjc/s10052-017-5317-4,NaN,NaN,10.1140/epjc/s10052-017-5317-4
5,1165GBP,1510,10.1038/s41598-017-12316-4,2045-2322,2045-2322,2045-2322,10.1038/s41598-017-12316-4,NaN,NaN,10.1038/s41598-017-12316-4
6,1880GBP,2437,10.1186/s12916-017-0968-4,1741-7015,1741-7015,1741-7015,10.1186/s12916-017-0968-4,NaN,NaN,10.1186/s12916-017-0968-4
7,0NOT,0,10.1140/epjc/s10052-017-5286-7,"1434-6044,1434-6052","1434-6044,1434-6052","1434-6044,1434-6052",10.1140/epjc/s10052-017-5286-7,NaN,NaN,10.1140/epjc/s10052-017-5286-7
8,550CHF,553,10.3390/foods6110094,2304-8158,2304-8158,2304-8158,10.3390/foods6110094,NaN,NaN,10.3390/foods6110094
9,0NOT,0,10.1007/jhep10(2017)180,1029-8479,1029-8479,1029-8479,10.1007/jhep10(2017)180,NaN,NaN,10.1007/jhep10(2017)180


In [ ]:
52

In [1]:
#cp gas.json gas3.json

In [45]:
rdlyc=pd.read_json('cites_refs_missing.json').sort_index().fillna('').reset_index(drop=True)

In [19]:
refs=get_cites_refs(browser,url,maxcites=65)

In [22]:
refs

"A longitudinal study assessing the maintenance condition of cadres of four types of wheelchairs provided in low-resource areas; K Rispin, K Riseling, J Wee\xa0- Disability and Rehabilitation\xa0…, 2018 - Taylor & Francis\nInnovations for Evaluation Research: Multiform Protocols, Visual Analog Scaling, and the Retrospective Pretest–Posttest Design; R Chang, TD Little\xa0- Evaluation & the health professions, 2018 - journals.sagepub.com\nWinning the war on state-sponsored propaganda; E Murrock, J Amulya, M Druckman, T Liubyva - irex.org\nVisual analogue mood scale scores in healthy young versus older adults; L Machado, LM Thompson, CHR Brett\xa0- International psychogeriatrics, 2018 - cambridge.org\nFOAM EM RSS; S Sick - foamem.com\nFOAM EM RSS; C McLeod - foamem.com\nToward a better measure: Integrating noncognitive constructs using Action-Control Theory; R Chang - 2016 - ttu-ir.tdl.org\nSelf-reported pain scales; C McLeod\xa0- Don't Forget The Bubbles, 2018 - dontforgetthebubbles.com\

In [23]:
rdlyc[rdlyc.GS_cites_link!=''].GS_cites_link[0]

'/scholar?cites=11746986714791086750&as_sdt=2005&sciodt=0,5&hl=en'

### Search GS citations refs for Redalyc

In [46]:
ini=143
maxcites=65
df=pd.DataFrame()
i=1
for url in rdlyc[rdlyc.GS_cites_link!=''].GS_cites_link[ini:]:
    print(i)
    i=i+1
    refs=get_cites_refs(browser,url,maxcites=maxcites)
    df=df.append({'GS_cites_link':url,'cites_refs':refs},
                ignore_index=True)
    df.to_json('cites_refs.json')
    time.sleep(random.uniform(0.9*t,1.1*t))

1
2
3
4
5


### Merge Redalyc cites with cites references

In [47]:
cr=pd.read_json('cites_refs.json')

rdlyc=rdlyc.fillna('')

rdlyc.shape

rdlyc['GS_quality_title']=rdlyc['GS_quality_title'].astype(float)

yy,nn=wp.df_split(rdlyc,on='GS_quality_title',Operator='>=',condition=0.9)

nn=nn.drop(['GS_DOI', 'GS_Journal', 'GS_Year', 'GS_abstract', 'GS_authors',
       'GS_cites', 'GS_cites_link', 'GS_profiles',  'GS_title'],axis='columns')

rdlyc=yy.append(nn).fillna('')

yy,nn=wp.df_split(rdlyc,on='GS_cites_link',Operator='!=',condition='')

cr=cr.rename({'cites_refs':'GS_cites_refs'},axis='columns')

yy=yy.merge(cr,on='GS_cites_link',how='left')

rdlyc=yy.append(nn).fillna('')

yy,nn=wp.df_split( rdlyc,on='GS_cites',Operator='!=',condition='' )

nn['GS_cites']=0

rdlyc=yy.append(nn)

rdlyc['GS_cites']=rdlyc.GS_cites.astype(int)

rdlyc.to_excel('redalyc_gs.xlsx',index=False)

### tmp

In [37]:
rdlyc[rdlyc.GS_cites_link!=''].GS_cites_link[ini:].shape

(148,)

In [ ]:
refs=get_cites_refs(browser,url,maxcites=65)

In [33]:
cp gsta.json gsta_missingdois.json

In [29]:
rd=pd.read_json('gas.json')

In [68]:
pd.DataFrame().append({},ignore_index=True)

""
0


See: https://stackoverflow.com/a/43413418/2268280

In [54]:
SAVERUN=False
if SAVERUN:
    dfgsy,dfgsn=wp.df_split( dfgs,on='title',Operator='!=',condition='')
    dfgsy.to_json('yellow1.json')
    dfgsn.to_json('yellowno.json')

array(['10.7705/biomedica.v37i3.4058', '10.17533/udea.esde.v73n162a06',
       '10.24201/hm.v64i4.3123', ..., '10.17533/udea.ikala.v19n3a05',
       '10.11144/javeriana.mavae9-2.eacv',
       '10.11144/javeriana.rgyps13-26.eaht'], dtype=object)

In [6]:
import pandas as pd

In [7]:
kk=pd.read_excel('gs_516_560.xlsx').append(pd.read_excel('gs_560_844'))

In [11]:
516+284

800

In [13]:
kk[1:284].to_excel('gs_516_800.xlsx',index=False)

In [24]:
adfgs=wp.fill_NaN(dfgs)

In [25]:
dfgs[:41].to_excel('gs1.xlsx',index=False)

In [8]:
dfgs[:50]

NameError: name 'dfgs' is not defined

# Consultar  DOAJ API con DOI
Ver: https://doaj.org/api/v1/docs#!/Search/get_api_v1_search_articles_search_query

## Carga navegador JSON

In [0]:
import pandas as pd

## Consulta

In [0]:
DOI='10.1016/j.physletb.2018.01.009'
df=pd.read_json('https://doaj.org/api/v1/search/articles/{}'.format(DOI))

## Análisis de resultado:
Nota: La celdas con estructuras anidadas (que comienzan con `[` ó `{`) sa analizan secuencialmente

In [0]:
df

,last,page,pageSize,query,results,timestamp,total
0,https://doaj.org/api/v1/search/articles/10.101...,1,10,10.1016/j.physletb.2018.01.009,"{u'last_updated': u'2018-03-05T12:09:16Z', u'i...",2018-1010T04:23:08Z,1


Desplegar colunna __results__:

In [0]:
p=pd.DataFrame(  df['results'].values[0] )
p

,bibjson,created_date,id,last_updated
abstract,The observation of neutrino oscillations estab...,2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
author,"[{u'affiliation': u'Physics Department, Univer...",2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
end_page,100,2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
identifier,"[{u'type': u'pissn', u'id': u'0370-2693'}, {u'...",2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
journal,"{u'publisher': u'Elsevier', u'language': [u'EN...",2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
link,[{u'url': u'http://www.sciencedirect.com/scien...,2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
month,3,2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
start_page,94,2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
subject,"[{u'code': u'QC1-999', u'term': u'Physics', u'...",2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
title,Expanding the reach of heavy neutrino searches...,2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z


Desplegar cada una de las celdas anidadas de la columna __bibjson__

In [0]:
pd.DataFrame(  p.loc['author','bibjson'] )

,affiliation,name
0,"Physics Department, Universidad de los Andes, ...",Andrés Flórez
1,"Department of Physics and Astronomy, Vanderbil...",Kaiwen Gui
2,"Department of Physics and Astronomy, Vanderbil...",Alfredo Gurrola
3,"Physics Department, Universidad de los Andes, ...",Carlos Patiño
4,"Department of Physics, Universidad de Antioqui...",Diego Restrepo


In [0]:
pd.DataFrame(  p.loc['identifier','bibjson'] )

,id,type
0,0370-2693,pissn
1,1873-2445,eissn
2,10.1016/j.physletb.2018.01.009,doi


In [0]:
 pd.DataFrame( [p.loc['journal','bibjson'] ] )


,country,issns,language,license,number,publisher,title,volume
0,NL,"[0370-2693, 1873-2445]",[EN],[{u'url': u'http://www.elsevier.com/journals/p...,C,Elsevier,Physics Letters B,778


In [0]:
pd.DataFrame( p.loc['journal','bibjson']['license'] )

,open_access,title,type,url
0,True,CC BY,CC BY,http://www.elsevier.com/journals/physics-lette...


In [0]:
pd.DataFrame(  p.loc['link','bibjson']  )

,type,url
0,fulltext,http://www.sciencedirect.com/science/article/p...


# Dask

In [0]:
! pip install dask cloudpickle > /dev/null

In [0]:
1+1

2

In [0]:
import dask as dk

In [0]:
from dask import dataframe

In [0]:
%%writefile kk.json
{"A":1,"B":3}
{"A":5,"B":6}

Writing kk.json


In [0]:
df=dataframe.read_json('kk.json')

In [0]:
df.compute()

,A,B
0,1,3
1,5,6


In [0]:
import pandas as pd

In [0]:
pd.read_json('kk.json',orient='records',lines=True)

,A,B
0,1,3
1,5,6


In [0]:
import os
os.chdir('drive/My Drive/Colab Notebooks/Data')

In [0]:
!head -n10 oagreenarticle.json > tmp.json

In [0]:
import pandas as pd

In [0]:
pd.read_json('tmp.json',orient='records',lines=True)[:3]

,best_oa_location,data_standard,doi,doi_url,genre,is_oa,journal_is_in_doaj,journal_is_oa,journal_issns,journal_name,oa_locations,published_date,publisher,title,updated,year,z_authors
0,{'url': 'https://doi.org/10.1016/j.ahj.2015.02...,2,10.1016/j.ahj.2015.02.019,https://doi.org/10.1016/j.ahj.2015.02.019,journal-article,True,False,False,0002-8703,American Heart Journal,[{'url': 'https://doi.org/10.1016/j.ahj.2015.0...,2015-06-01,Elsevier BV,Biomarkers for risk stratification of patients...,2018-06-18T19:10:12.692979,2015.0,"[{'given': 'Matthijs A.', 'family': 'Velders'}..."
1,{'url': 'https://doi.org/10.1021/acsnano.7b049...,2,10.1021/acsnano.7b04994,https://doi.org/10.1021/acsnano.7b04994,journal-article,True,False,False,"1936-0851,1936-086X",ACS Nano,[{'url': 'https://doi.org/10.1021/acsnano.7b04...,2017-09-08,American Chemical Society (ACS),Hybrid Surface Patterns Mimicking the Design o...,2018-06-14T11:39:08.680219,2017.0,[{'ORCID': 'http://orcid.org/0000-0001-5512-74...
2,{'url': 'https://doi.org/10.1016/s0925-4773(00...,2,10.1016/s0925-4773(00)00472-x,https://doi.org/10.1016/s0925-4773(00)00472-x,journal-article,True,False,False,0925-4773,Mechanisms of Development,[{'url': 'https://doi.org/10.1016/s0925-4773(0...,2000-12-01,Elsevier BV,Embryonic expression of a P2X 3 receptor encod...,2018-06-18T04:15:29.411710,2000.0,"[{'given': 'William H.J', 'family': 'Norton'},..."


In [0]:
tmp=dataframe.read_json('tmp.json')

In [0]:
tmp.compute()

,best_oa_location,data_standard,doi,doi_url,genre,is_oa,journal_is_in_doaj,journal_is_oa,journal_issns,journal_name,oa_locations,published_date,publisher,title,updated,year,z_authors
0,{'url': 'https://doi.org/10.1016/j.ahj.2015.02...,2,10.1016/j.ahj.2015.02.019,https://doi.org/10.1016/j.ahj.2015.02.019,journal-article,True,False,False,0002-8703,American Heart Journal,[{'url': 'https://doi.org/10.1016/j.ahj.2015.0...,2015-06-01,Elsevier BV,Biomarkers for risk stratification of patients...,2018-06-18T19:10:12.692979,2015.0,"[{'given': 'Matthijs A.', 'family': 'Velders'}..."
1,{'url': 'https://doi.org/10.1021/acsnano.7b049...,2,10.1021/acsnano.7b04994,https://doi.org/10.1021/acsnano.7b04994,journal-article,True,False,False,"1936-0851,1936-086X",ACS Nano,[{'url': 'https://doi.org/10.1021/acsnano.7b04...,2017-09-08,American Chemical Society (ACS),Hybrid Surface Patterns Mimicking the Design o...,2018-06-14T11:39:08.680219,2017.0,[{'ORCID': 'http://orcid.org/0000-0001-5512-74...
2,{'url': 'https://doi.org/10.1016/s0925-4773(00...,2,10.1016/s0925-4773(00)00472-x,https://doi.org/10.1016/s0925-4773(00)00472-x,journal-article,True,False,False,0925-4773,Mechanisms of Development,[{'url': 'https://doi.org/10.1016/s0925-4773(0...,2000-12-01,Elsevier BV,Embryonic expression of a P2X 3 receptor encod...,2018-06-18T04:15:29.411710,2000.0,"[{'given': 'William H.J', 'family': 'Norton'},..."
3,{'url': 'https://doi.org/10.1016/0021-8693(88)...,1,10.1016/0021-8693(88)90058-0,https://doi.org/10.1016/0021-8693(88)90058-0,journal-article,True,False,False,0021-8693,Journal of Algebra,[{'url': 'https://doi.org/10.1016/0021-8693(88...,1988-10-01,Elsevier BV,Corrigendum,2018-06-18T04:18:43.222135,1988.0,None
4,{'url': 'https://doi.org/10.1016/j.mod.2006.03...,2,10.1016/j.mod.2006.03.003,https://doi.org/10.1016/j.mod.2006.03.003,journal-article,True,False,False,0925-4773,Mechanisms of Development,[{'url': 'https://doi.org/10.1016/j.mod.2006.0...,2006-05-01,Elsevier BV,Impaired development of the Harderian gland in...,2018-06-18T04:20:45.498171,2006.0,"[{'given': 'Andreas', 'family': 'Schild'}, {'g..."
5,{'url': 'https://doi.org/10.1103/physrevc.96.0...,2,10.1103/physrevc.96.014310,https://doi.org/10.1103/physrevc.96.014310,journal-article,True,False,False,"2469-9985,2469-9993",Physical Review C,[{'url': 'https://doi.org/10.1103/physrevc.96....,2017-07-14,American Physical Society (APS),Nuclear deformation in the A≈100 region: Compa...,2018-06-16T19:57:39.403083,2017.0,"[{'given': 'A.', 'family': 'de Roubin'}, {'giv..."
6,{'url': 'https://doi.org/10.1016/s0960-9822(07...,2,10.1016/s0960-9822(07)00493-9,https://doi.org/10.1016/s0960-9822(07)00493-9,journal-article,True,False,False,0960-9822,Current Biology,[{'url': 'https://doi.org/10.1016/s0960-9822(0...,1998-10-01,Elsevier BV,Protein kinase B (PKB/Akt) activity is elevate...,2018-06-18T04:21:43.465040,1998.0,"[{'given': 'Daphne', 'family': 'Haas-Kogan'}, ..."
7,{'url': 'https://doi.org/10.1016/s1470-2045(13...,2,10.1016/s1470-2045(13)70163-3,https://doi.org/10.1016/s1470-2045(13)70163-3,journal-article,True,False,False,1470-2045,The Lancet Oncology,[{'url': 'https://doi.org/10.1016/s1470-2045(1...,2013-07-01,Elsevier BV,Panitumumab and irinotecan versus irinotecan a...,2018-06-18T19:11:38.984437,2013.0,"[{'given': 'Matthew T', 'family': 'Seymour'}, ..."
8,{'url': 'https://doi.org/10.24159/joec.2014.20...,2,10.24159/joec.2014.20.4.281,https://doi.org/10.24159/joec.2014.20.4.281,journal-article,True,False,False,1598-9283,Journal of Education & Culture,[{'url': 'https://doi.org/10.24159/joec.2014.2...,2014-12-01,Education Research Institute,The Research on the Values and Lifestyles of K...,2018-06-18T05:26:34.731357,2014.0,"[{'family': 'Kim,Heung-Kyu'}, {'family': '이상란'}]"
9,{'url': 'https://doi.org/10.5194/acp-2016-560-...,2,10.5194/acp-2016-560-ac1,https://doi.org/10.5194/acp-2016-560-ac1,component,True,False,False,None,None,[{'url': 'https://doi.org/10.5194/acp-2016-560...,None,Copernicus GmbH,None,2018-

In [0]:
dataframe.read_json?

In [0]:
df=dataframe.read_json('oagreenarticle.json')

NameError: ignored

In [0]:
import dask.bag as db

In [0]:
import json
b = db.read_text('tmp.json').map(json.loads)

In [0]:
bb=b.to_dataframe()

In [0]:
bbb=bb['doi']

In [2]:
from webdriver_manager.firefox import GeckoDriverManager

from selenium import webdriver

ls -l /home/restrepo/.wdm/geckodriver/v0.23.0/linux64/geckodriver

driver = webdriver.Firefox(executable_path=
                '/home/restrepo/.wdm/geckodriver/v0.23.0/linux64/geckodriver')
#reinstall path: GeckoDriverManager().install())

In [16]:
#driver.set_page_load_timeout(30)

In [17]:
url='https://scholar.google.com.co/scholar?hl=en&as_sdt=0%2C5&q=10.1590%2Fs0103-84782009000100018'

In [22]:
driver.get('http://google.com')

In [20]:
driver.page_source

'<html><head><meta http-equiv="content-type" content="text/html; charset=utf-8"><meta name="viewport" content="initial-scale=1"><title>https://scholar.google.com.co/scholar?hl=en&amp;as_sdt=0%2C5&amp;q=10.1590%2Fs0103-84782009000100018</title></head>\n<body style="font-family: arial, sans-serif; background-color: #fff; color: #000; padding:20px; font-size:18px;" onload="e=document.getElementById(\'captcha\');if(e){e.focus();}">\n<div style="max-width:400px;">\n<hr style="color:#ccc; background-color:#ccc;" size="1" noshade=""><br>\n<form id="captcha-form" action="index" method="post">\n<script type="text/javascript" async="" src="https://www.gstatic.com/recaptcha/api2/v1540189908068/recaptcha__en.js"></script><script src="https://www.google.com/recaptcha/api.js" async="" defer=""></script>\n<script>var submitCallback = function(response) {document.getElementById(\'captcha-form\').submit();};</script>\n<div id="recaptcha" class="g-recaptcha" data-sitekey="6LfwuyUTAAAAAOAmoS0fdqijC2PbbdH

In [8]:
browser=driver

In [20]:
browser.get('https://github.com/mozilla/geckodriver')

InvalidSessionIdException: Message: Tried to run command without establishing a connection


In [1]:
import pandas as pd

In [4]:
gsta=pd.read_excel('gsta_full.xlsx')